# R Kernel on Vertex AI Workbench Instances

How to use R on a Vertex AI Workbench Instance

## Setup

1.  Create a new vertex ai workbench instance  [https://cloud.google.com/vertex-ai/docs/workbench/instances/introduction](https://cloud.google.com/vertex-ai/docs/workbench/instances/introduction)     
2.  Open a terminal session from the Launcher     
3.  enter commands below in the terminal (one line at a time)
 
  
```sh
ENV_NAME=r-env
conda create -n $ENV_NAME r-essentials r-base rpy2
conda activate $ENV_NAME
python -m ipykernel install --user --name=$ENV_NAME
```
  
4.  Return to the Launcher screen and click the R kernel    


## Install R packages

In [ ]:
required_packages <- c("bigrquery", "googleCloudStorageR" , "glue", "httr")
install.packages(setdiff(required_packages, rownames(installed.packages())))

## set constants 

In [ ]:
email <- "your-email@your-company-name.com"
project_id <- "your-project-id"
## try to set from gcloud if not specified 
if (email == "your-email@your-company-name.com") {
    email <- system("gcloud config get-value core/account", intern = TRUE)}
if (project_id == "your-project-id"){
    project_id <- system("gcloud config get-value project", intern = TRUE)}

In [ ]:
print(email)

In [ ]:
print(project_id)

## load R packages 

In [ ]:
library(googleCloudStorageR)
library(bigrquery)
library(gargle)
# set when debugging
# options(gargle_verbosity = "debug") 

## Authenticate Clients

In [ ]:
## set scope to enable access for each R package and use token based auth
## to simplify
### GCS - https://code.markedmondson.me/googleCloudStorageR/articles/googleCloudStorageR.html#token-authentication-1
### BQ - https://bigrquery.r-dbi.org/reference/bq_auth.html
scope <- c("https://www.googleapis.com/auth/cloud-platform")
token <- token_fetch(scopes = scope, email = email)

In [ ]:
## authenticate with each service
gcs_auth(token = token)
bq_auth(token = token)

## Test connections

### List Google Cloud Storage Buckets

In [ ]:
buckets <- gcs_list_buckets(project_id)
buckets

### List bigquery datasets 

In [ ]:
datasets <- as.data.frame(do.call(rbind, bq_project_datasets(project_id)))
datasets